# DWthon 9.0 -  Electricity price forecastingday-ahead 

In [49]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import catboost as ctb
import lightgbm as lgb

import eli5

import matplotlib.pyplot as plt
from matplotlib.ticker import  MultipleLocator

In [50]:
df = pd.read_csv("../input/energy_price.csv", parse_dates=["datetime"])
df["date"] = df["datetime"].dt.date

In [51]:
df.head()

,datetime,price,is_test,date
0,2019-06-28 00:00:00,220.54,False,2019-06-28
1,2019-06-28 01:00:00,203.85,False,2019-06-28
2,2019-06-28 02:00:00,203.85,False,2019-06-28
3,2019-06-28 03:00:00,203.85,False,2019-06-28
4,2019-06-28 04:00:00,203.85,False,2019-06-28


In [52]:
df["datetime_b24h"] = df["datetime"].shift(24)
is_valid_datetime_set = set(df[ df["is_test"] ]["datetime_b24h"])
df["is_valid"] = df["datetime"].map(lambda x: x in is_valid_datetime_set)


df["datetime_b48h"] = df["datetime"].shift(48)
is_train_datetime_set = set(df[ df["is_test"] ]["datetime_b48h"])
df["is_train"] = df["datetime"].map(lambda x: x in is_train_datetime_set)

In [53]:
df.head()

,datetime,price,is_test,date,datetime_b24h,is_valid,datetime_b48h,is_train
0,2019-06-28 00:00:00,220.54,False,2019-06-28,NaT,False,NaT,False
1,2019-06-28 01:00:00,203.85,False,2019-06-28,NaT,False,NaT,False
2,2019-06-28 02:00:00,203.85,False,2019-06-28,NaT,False,NaT,False
3,2019-06-28 03:00:00,203.85,False,2019-06-28,NaT,False,NaT,False
4,2019-06-28 04:00:00,203.85,False,2019-06-28,NaT,False,NaT,False


In [54]:
def mape(y_test, y_pred):
    return np.mean( np.abs(y_test - y_pred) / y_test * 100 )

def run_experiment(df, model, fe_func, feats):
    df_fe = fe_func(df.copy())
    
    df_train = df_fe[ df_fe["is_train"] ].copy()
    df_valid = df_fe[ df_fe["is_valid"] ].copy()
    
    X_train = df_train[feats].values
    y_train = df_train["price"].values

    X_valid = df_valid[feats].values
    y_valid = df_valid["price"].values

    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    
    print( f"MAPE: {mape(y_valid, y_pred)}%" )
    print(feats)
    
    return eli5.show_weights(model, feature_names=feats, top=len(feats))



def predict_and_save_submit_file(filename, df, model, fe_func, feats):
    df_fe = fe_func(df.copy())
    
    df_valid = df_fe[ df_fe["is_valid"] ].copy()
    df_test = df_fe[ df_fe["is_test"] ].copy()
    
    X_valid = df_valid[feats].values
    y_valid = df_valid["price"].values
    
    X_test = df_test[feats].values
    
    model.fit(X_valid, y_valid)
    
    df_test["price"] = model.predict(X_test)
    
    output_path = f"../output/{filename}.csv"
    df_test[ ["datetime", "price"] ].to_csv(output_path, index=False)
    print(f"Saved: {output_path}")

Мы начнем с простых экспериментов.


Определим функцию `fe_func`, которая отвечает за создание новых признаков.

In [55]:
def fe_func(df):
    df["year"] = df["datetime"].dt.year
    df["month"] = df["datetime"].dt.month
    df["dayofweek"] = df["datetime"].dt.dayofweek
    df["hour"] = df["datetime"].dt.hour
            
    
    for offset in range(1, 8):
        df[f"shift_price_{offset}d"] = df["price"].shift(24*offset)
        
    return df

In [72]:
def fe_func(df):
    df["day"] = df["datetime"].dt.day
    df["dayofweek"] = df["datetime"].dt.dayofweek
    df["week"] = df["datetime"].dt.week
    
    df['dayofyear'] = df['datetime'].dt.dayofyear
    df["month"] = df["datetime"].dt.month
    df["year"] = df["datetime"].dt.year
    
    df['quarter'] = df['datetime'].dt.quarter
    df['hour'] =  df['datetime'].dt.hour
    df["is_weekend"] = df["dayofweek"].isin([5, 6])
    
    df['pay_per_day'] = df['hour'].map(lambda x: 1 if ((x >=6) & (x<13)) | ((x >=15) & (x<22)) else 0 )
    df['dark_hour'] = df['hour'].map(lambda x: 1 if (x >=23) & (x<=6) else 0 )
        
    df['holidays'] = df.month.map(lambda x: 1 if (x == 7) | (x == 8) else 0).astype('int')
     
    df['hour_5'] = df.hour.isin([5])
    df['hour_2'] = df.hour.isin([2])
    df['hour_4'] = df.hour.isin([4])
    df['hour_23'] = df.hour.isin([23])
    df['hour_0'] = df.hour.isin([0])
    df['hour_1'] = df.hour.isin([1])
    df['hour_3'] = df.hour.isin([3])
    df['hour_6'] = df.hour.isin([6])
    df['hour_7'] = df.hour.isin([7])
    df['hour_8'] = df.hour.isin([8])
    df['hour_9'] = df.hour.isin([9])
    df['hour_10'] = df.hour.isin([10])
    df['hour_11'] = df.hour.isin([11])
    df['hour_12'] = df.hour.isin([12])
    df['hour_13'] = df.hour.isin([13])
    df['hour_14'] = df.hour.isin([14])
    df['hour_15'] = df.hour.isin([15])
    df['hour_16'] = df.hour.isin([16])
    df['hour_17'] = df.hour.isin([17])
    df['hour_18'] = df.hour.isin([18])
    df['hour_19'] = df.hour.isin([19])
    df['hour_20'] = df.hour.isin([20])
    df['hour_21'] = df.hour.isin([21])
    df['hour_22'] = df.hour.isin([22])
    
    #########
            
    
    for offset in range(1, 8):
        df[f"shift_price_{offset}d"] = df["price"].shift(24*offset)
        
    return df

### `LinearRegression`

In [73]:
model = LinearRegression()

shifted_feats = [
    "shift_price_1d", "shift_price_2d", "shift_price_3d", 
    "shift_price_4d"
]
feats = ["hour", "dayofweek"] + shifted_feats

run_experiment(df, model, fe_func, feats)

MAPE: 20.472355068081693%
['hour', 'dayofweek', 'shift_price_1d', 'shift_price_2d', 'shift_price_3d', 'shift_price_4d']


In [74]:
predict_and_save_submit_file("simple_ln", df, model, fe_func, feats)

Saved: ../output/simple_ln.csv


Теперь проверим на разных моделях:
- деревья решений
- случайные леса
- xgboost
- catboost
- ... можешь проверить еще больше 

In [75]:
shifted_feats = [
    "shift_price_1d", "shift_price_2d", "shift_price_3d", 
    "shift_price_4d"
]
feats_simple = ["hour", "dayofweek"] + shifted_feats

### `DecisionTreeRegressor`

In [76]:
model = DecisionTreeRegressor(max_depth=7, random_state=0)
run_experiment(df, model, fe_func, feats_simple)

MAPE: 22.287783349097555%
['hour', 'dayofweek', 'shift_price_1d', 'shift_price_2d', 'shift_price_3d', 'shift_price_4d']


Weight,Feature
0.6846,shift_price_1d
0.1716,shift_price_3d
0.0713,shift_price_4d
0.0345,shift_price_2d
0.0266,dayofweek
0.0114,hour


### `RandomForestRegressor`

In [77]:
model = RandomForestRegressor(max_depth=10, n_estimators=200, random_state=0)
run_experiment(df, model, fe_func, feats_simple)

MAPE: 20.30216029056043%
['hour', 'dayofweek', 'shift_price_1d', 'shift_price_2d', 'shift_price_3d', 'shift_price_4d']


Weight,Feature
0.3910 ± 0.5030,shift_price_1d
0.3568 ± 0.5396,shift_price_2d
0.1578 ± 0.1937,shift_price_3d
0.0472 ± 0.0523,shift_price_4d
0.0276 ± 0.0279,hour
0.0195 ± 0.0217,dayofweek


### `XGBRegressor`

In [78]:
model = xgb.XGBRegressor(max_depth=10, n_estimators=300, learning_rate=0.3, random_state=0)
run_experiment(df, model, fe_func, feats_simple)

MAPE: 21.482499422214634%
['hour', 'dayofweek', 'shift_price_1d', 'shift_price_2d', 'shift_price_3d', 'shift_price_4d']


Weight,Feature
0.3488,shift_price_2d
0.3348,shift_price_1d
0.2123,shift_price_3d
0.0468,shift_price_4d
0.0438,dayofweek
0.0134,hour


### `CatBoostRegressor`

In [79]:
model = ctb.CatBoostRegressor(max_depth=7, n_estimators=500, learning_rate=0.15, verbose=False)
run_experiment(df, model, fe_func, feats_simple)

MAPE: 20.22811244610589%
['hour', 'dayofweek', 'shift_price_1d', 'shift_price_2d', 'shift_price_3d', 'shift_price_4d']


Weight,Feature
0.2572,shift_price_3d
0.2510,shift_price_1d
0.1721,shift_price_2d
0.1350,hour
0.0981,shift_price_4d
0.0866,dayofweek


In [80]:
df.groupby(
    df['datetime'].dt.date
)['price'].transform('median')

0        236.79
1        236.79
2        236.79
3        236.79
4        236.79
          ...  
26968    480.96
26969    480.96
26970    480.96
26971    480.96
26972    480.96
Name: price, Length: 26973, dtype: float64

In [81]:
offset = 1

df[f"price_hist_m{offset}d_median"] = df.groupby(
    df['datetime'].dt.date
)['price'].transform('median').shift(24*offset)

In [91]:
def fe_func_v2(df):
    df["year"] = df["datetime"].dt.year
    df["month"] = df["datetime"].dt.month
    df["day"] = df["datetime"].dt.day
    df["dayofweek"] = df["datetime"].dt.dayofweek
    df["hour"] = df["datetime"].dt.hour
    df["is_weekend"] = df["dayofweek"].isin([5, 6])

In [92]:
def fe_func_v2(df):
    df["day"] = df["datetime"].dt.day
    df["dayofweek"] = df["datetime"].dt.dayofweek
    df["week"] = df["datetime"].dt.week
    
    df['dayofyear'] = df['datetime'].dt.dayofyear
    df["month"] = df["datetime"].dt.month
    df["year"] = df["datetime"].dt.year
    
    df['quarter'] = df['datetime'].dt.quarter
    df['hour'] =  df['datetime'].dt.hour
    df["is_weekend"] = df["dayofweek"].isin([5, 6])
    
    df['pay_per_day'] = df['hour'].map(lambda x: 1 if ((x >=6) & (x<13)) | ((x >=15) & (x<22)) else 0 )
    df['dark_hour'] = df['hour'].map(lambda x: 1 if (x >=23) & (x<=6) else 0 )
        
    df['holidays'] = df.month.map(lambda x: 1 if (x == 7) | (x == 8) else 0).astype('int')
     
    df['hour_5'] = df.hour.isin([5])
    df['hour_2'] = df.hour.isin([2])
    df['hour_4'] = df.hour.isin([4])
    df['hour_23'] = df.hour.isin([23])
    df['hour_0'] = df.hour.isin([0])
    df['hour_1'] = df.hour.isin([1])
    df['hour_3'] = df.hour.isin([3])
    df['hour_6'] = df.hour.isin([6])
    df['hour_7'] = df.hour.isin([7])
    df['hour_8'] = df.hour.isin([8])
    df['hour_9'] = df.hour.isin([9])
    df['hour_10'] = df.hour.isin([10])
    df['hour_11'] = df.hour.isin([11])
    df['hour_12'] = df.hour.isin([12])
    df['hour_13'] = df.hour.isin([13])
    df['hour_14'] = df.hour.isin([14])
    df['hour_15'] = df.hour.isin([15])
    df['hour_16'] = df.hour.isin([16])
    df['hour_17'] = df.hour.isin([17])
    df['hour_18'] = df.hour.isin([18])
    df['hour_19'] = df.hour.isin([19])
    df['hour_20'] = df.hour.isin([20])
    df['hour_21'] = df.hour.isin([21])
    df['hour_22'] = df.hour.isin([22])
    
    #########

    
    offsets = list(range(1, 8)) + [14, 28]
    for offset in offsets:
        df[f"shift_price_{offset}d"] = df["price"].shift(24*offset)
        
        for agg_func in ["mean", "median"]:
            df[f"price_hist_m{offset}d_{agg_func}"] = df.groupby(
                df['datetime'].dt.date
            )['price'].transform(agg_func).shift(24*offset)
        
        

        
    shifted_feats = [x for x in df.columns if "shift_price_" in x]
    df["shift_price_mean"] = df[shifted_feats].mean(axis=1)
    df["shift_price_median"] = df[shifted_feats].median(axis=1)
    df["shift_price_max"] = df[shifted_feats].max(axis=1)
    df["shift_price_min"] = df[shifted_feats].min(axis=1)
    df["shift_price_prc90"] = df[shifted_feats].agg(lambda x: np.percentile(x, 90), axis=1)
    df["shift_price_prc10"] = df[shifted_feats].agg(lambda x: np.percentile(x, 10), axis=1)
    df["shift_price_prc75"] = df[shifted_feats].agg(lambda x: np.percentile(x, 75), axis=1)
    df["shift_price_prc25"] = df[shifted_feats].agg(lambda x: np.percentile(x, 25), axis=1)    
    
    df["shift_price_diff_25"] = df["shift_price_prc75"] - df["shift_price_prc25"]
    df["shift_price_diff"] = df["shift_price_max"] - df["shift_price_min"]
    df["shift_price_diff_10"] = df["shift_price_prc90"] - df["shift_price_prc10"]
    df["shift_price_diff_10_25"] = df["shift_price_diff_10"] - df["shift_price_diff_25"] 
    
    
        
    return df

In [101]:
shifted_feats = [
    "shift_price_1d", "shift_price_2d", "shift_price_3d",
    "shift_price_4d", "shift_price_5d", "shift_price_6d",
    "shift_price_7d", "shift_price_14d", "shift_price_28d",
    
    "shift_price_mean", 
    "shift_price_prc90", "shift_price_prc25", "shift_price_prc75",
    "shift_price_diff", "shift_price_diff_10", "shift_price_diff_25",
    "shift_price_diff_10_25"
    
]

hist_feats = [
    "price_hist_m3d_median", "price_hist_m1d_median", "price_hist_m14d_median",
    "price_hist_m1d_mean", "price_hist_m2d_mean",
]

feats_v2 = (
    ["is_weekend", "dayofweek", "month"] 
    + shifted_feats 
    + hist_feats
)

In [102]:
feats_v2 = (
    ["day", "dayofweek", "week", "dayofyear", "month", "year", 
     "quarter", "hour", "is_weekend", "pay_per_day", 
    "holidays", "hour_1", "hour_2", "hour_3", "hour_4", "hour_5", 
     "hour_6", "hour_7", "hour_8", "hour_9", "hour_10", "hour_11", 
     "hour_12", "hour_13", "hour_14", "hour_16", "hour_16", "hour_17", 
     "hour_18", "hour_19", "hour_20", "hour_21", "hour_22", "hour_23" ]
    + shifted_feats 
    + hist_feats
)

In [113]:
shifted_feats = [
    "shift_price_1d", "shift_price_2d", "shift_price_3d",
    "shift_price_4d", "shift_price_5d", "shift_price_6d",
    "shift_price_7d", "shift_price_14d", "shift_price_28d",
    
    "shift_price_mean", 
    "shift_price_prc90", "shift_price_prc25", "shift_price_prc75",
    "shift_price_diff", "shift_price_diff_10", "shift_price_diff_25",
    "shift_price_diff_10_25",
    
    "shift_price_mean", "shift_price_median", "shift_price_max", 
    "shift_price_min", "shift_price_prc90", "shift_price_prc10",
    "shift_price_prc75", "shift_price_prc25"
    
        
]

hist_feats = [
    "price_hist_m3d_median", "price_hist_m1d_median", "price_hist_m14d_median",
    "price_hist_m1d_mean", "price_hist_m2d_mean",
]
feats_v2 = (
    ["day", "dayofweek", "week", "dayofyear", "month", "year", 
     "quarter", "hour", "is_weekend", "pay_per_day", 
    "holidays", "hour_1", "hour_2", "hour_3", "hour_4", "hour_5", 
     "hour_6", "hour_7", "hour_8", "hour_9", "hour_10", "hour_11", 
     "hour_12", "hour_13", "hour_14", "hour_16", "hour_16", "hour_17", 
     "hour_18", "hour_19", "hour_20", "hour_21", "hour_22", "hour_23" ]
    + shifted_feats 
    + hist_feats
)

In [114]:
model = ctb.CatBoostRegressor(
    max_depth=7, n_estimators=500, learning_rate=0.15, 
    verbose=False, random_seed=0
)


run_experiment(df, model, fe_func_v2, feats_v2)

MAPE: 19.89751945763451%
['day', 'dayofweek', 'week', 'dayofyear', 'month', 'year', 'quarter', 'hour', 'is_weekend', 'pay_per_day', 'holidays', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_16', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'shift_price_1d', 'shift_price_2d', 'shift_price_3d', 'shift_price_4d', 'shift_price_5d', 'shift_price_6d', 'shift_price_7d', 'shift_price_14d', 'shift_price_28d', 'shift_price_mean', 'shift_price_prc90', 'shift_price_prc25', 'shift_price_prc75', 'shift_price_diff', 'shift_price_diff_10', 'shift_price_diff_25', 'shift_price_diff_10_25', 'shift_price_mean', 'shift_price_median', 'shift_price_max', 'shift_price_min', 'shift_price_prc90', 'shift_price_prc10', 'shift_price_prc75', 'shift_price_prc25', 'price_hist_m3d_median', 'price_hist_m1d_median', 'price_hist_m14d_median', 'price_hist_m1d_mean', 'price_his

Weight,Feature
0.0873,shift_price_1d
0.0826,price_hist_m2d_mean
0.0668,shift_price_mean
0.0617,shift_price_prc25
0.0529,shift_price_6d
0.0486,price_hist_m3d_median
0.0387,shift_price_prc75
0.0369,shift_price_7d
0.0365,day
0.0358,shift_price_diff_25


In [115]:
predict_and_save_submit_file("fe_ctb_five_5", df, model, fe_func_v2, feats_v2)

Saved: ../output/fe_ctb_five_5.csv


In [37]:
df_tmp = df[ 
    (df["date"] >= pd.to_datetime("2020-02-01") )
    &
    (df["date"] < pd.to_datetime("2020-02-05") )
].copy()

df_tmp.head(5)

,datetime,price,is_test,date,datetime_b24h,is_valid,datetime_b48h,is_train
5232,2020-02-01 00:00:00,101.03,False,2020-02-01,2020-01-31 00:00:00,False,2020-01-30 00:00:00,False
5233,2020-02-01 01:00:00,77.00,False,2020-02-01,2020-01-31 01:00:00,False,2020-01-30 01:00:00,False
5234,2020-02-01 02:00:00,72.00,False,2020-02-01,2020-01-31 02:00:00,False,2020-01-30 02:00:00,False
5235,2020-02-01 03:00:00,71.21,False,2020-02-01,2020-01-31 03:00:00,False,2020-01-30 03:00:00,False
5236,2020-02-01 04:00:00,70.00,False,2020-02-01,2020-01-31 04:00:00,False,2020-01-30 04:00:00,False


In [38]:
(101.03 + 77.00) / 2

89.015

In [39]:
df_tmp["price"].rolling(2).agg("mean").head(2)

5232       NaN
5233    89.015
Name: price, dtype: float64

In [40]:
(101.03 + 77.00 + 72.00) / 3

83.34333333333333

In [41]:
df_tmp["price"].rolling(3).agg("mean").head(3)

5232          NaN
5233          NaN
5234    83.343333
Name: price, dtype: float64

In [42]:
df_tmp["rolling24_mean"] = df_tmp["price"].rolling(24).agg("mean")
df_tmp.head(48)

,datetime,price,is_test,date,datetime_b24h,is_valid,datetime_b48h,is_train,rolling24_mean
5232,2020-02-01 00:00:00,101.03,False,2020-02-01,2020-01-31 00:00:00,False,2020-01-30 00:00:00,False,NaN
5233,2020-02-01 01:00:00,77.00,False,2020-02-01,2020-01-31 01:00:00,False,2020-01-30 01:00:00,False,NaN
5234,2020-02-01 02:00:00,72.00,False,2020-02-01,2020-01-31 02:00:00,False,2020-01-30 02:00:00,False,NaN
5235,2020-02-01 03:00:00,71.21,False,2020-02-01,2020-01-31 03:00:00,False,2020-01-30 03:00:00,False,NaN
5236,2020-02-01 04:00:00,70.00,False,2020-02-01,2020-01-31 04:00:00,False,2020-01-30 04:00:00,False,NaN
5237,2020-02-01 05:00:00,71.38,False,2020-02-01,2020-01-31 05:00:00,False,2020-01-30 05:00:00,False,NaN
5238,2020-02-01 06:00:00,73.00,False,2020-02-01,2020-01-31 06:00:00,False,2020-01-30 06:00:00,False,NaN
5239,2020-02-01 07:00:00,70.00,False,2020-02-01,2020-01-31 07:00:00,False,2020-01-30 07:00:00,False,NaN
5240,2020-02-01 08:00:00,80.49,False,2020-02-01,2020-01-31 08:00:00,False,2020-01-30 08:00:00,False,NaN
5241,2020-02-01 09:00:00,108.00,False,2020-02-01,2020-01-31 09:00:00,False,2020-01-30 09:00:00,False,NaN


In [43]:
df[  df["date"] == pd.to_datetime("2020-02-01") ]["price"].mean()

121.43